In [1]:
from pathlib import Path

import shutil

In [2]:
# ----------

def verify_path(path):
    return Path(path).exists()

def verify_and_create_path(path):
    path = Path(path)
    if not path.exists():
        path.mkdir(parents=True, exist_ok=True)

# ----------

In [3]:
EXP_PATH = '../experiments'

SLIM_EXP_PATH = '../slim_experiments'
verify_and_create_path(SLIM_EXP_PATH)

In [ ]:
models = [f for f in Path(EXP_PATH).iterdir() if f.is_dir()]
print(models)

In [ ]:
for model in models:
    path_valuations = model / "valuations"

    actions = [f.name for f in path_valuations.iterdir() if f.is_dir()]

    for action in actions:
        path_action = path_valuations / action

        all_subfolders = [f for f in path_action.rglob("*") if f.is_dir()]
        if all_subfolders:
            max_depth = max(len(f.relative_to(path_action).parts) for f in all_subfolders)

            if action == "clean_accuracy":
                target_depth = max_depth
            elif action == "adversarial":
                target_depth = max_depth - 1
            else:
                continue

            filtered_folders = [f for f in all_subfolders if len(f.relative_to(path_action).parts) == target_depth]
        else:
            filtered_folders = []

        for filtered_folder in filtered_folders:
            filtered_folder_path = Path(filtered_folder)

            relative_path = filtered_folder_path.relative_to(EXP_PATH)

            new_complete_path = Path(SLIM_EXP_PATH) / relative_path

            if not verify_path(new_complete_path):
                verify_and_create_path(new_complete_path)

                for json_file in filtered_folder_path.rglob("*.json"):
                    dest_file = new_complete_path / json_file.name
                    shutil.copy(json_file, dest_file)

                    print(f"Copied {json_file} -> {dest_file}")